In [1]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self,match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        # self.teamNumbering = self.get_teamNumbering()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.team = self.get_team(team = 1)
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.dismissals_df = self.get_dismissals_df(0)
        self.dream11_points = self.get_dream11_points()
        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_team(self, team):
        team = []
        for i in self.json_api['rosters'][0]['roster']:
             team.append(i['athlete']['name'])
        return team
    
    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_dismissals_df(self, team):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        for i in self.json_api['rosters'][team]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][0]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][0]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][0]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        # a = pd.read_json(m.match_url)
        return df
    



In [2]:
matchNumber = 1426290

m = Match(matchNumber)


/tmp/ipykernel_792507/1431624738.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_792507/1431624738.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_792507/1431624738.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [7]:
m.away_team

{'name': 'GT', 'fullName': 'Gujarat Titans', 'id': '1298769'}

In [3]:
m.team1_batting_df

# m.team2_bowling_df

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Wriddhiman Saha,c †Karthik b Mohammed Siraj,1,7,0,0,14.29
1,Shubman Gill,c Vyshak b Mohammed Siraj,2,7,0,0,28.57
2,Sai Sudharsan,c Kohli b Green,6,14,0,0,42.86
3,M Shahrukh Khan,run out (Kohli),37,24,5,1,154.17
4,David Miller,c Maxwell b Sharma,30,20,3,2,150.00
5,Rahul Tewatia,c Vyshak b Yash Dayal,35,21,5,1,166.67
6,Rashid Khan,b Yash Dayal,18,14,2,1,128.57
7,Vijay Shankar,c Mohammed Siraj b Vyshak,10,7,2,0,142.86
8,Manav Suthar,c Swapnil Singh b Vyshak,1,2,0,0,50.00
9,Mohit Sharma,run out (†Karthik/Vyshak),0,1,0,0,0.00


In [4]:
x = m.team2_batting_df

x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
x['Fantasy Points'] = numpy.where(x['Runs'] > 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])
x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10), x['Fantasy Points'] - 2 , x['Fantasy Points'])

y = m.team1_bowling_df

y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])

m.team2_batting_df


,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate,Fantasy Points
0,Virat Kohli,c †Saha b Noor Ahmad,42,27,2,4,155.56,52
1,Faf du Plessis,c Shahrukh Khan b Little,64,23,10,3,278.26,88
2,Will Jacks,c Shahrukh Khan b Noor Ahmad,1,3,0,0,33.33,1
3,Rajat Patidar,c Miller b Little,2,3,0,0,66.67,2
4,Glenn Maxwell,c Miller b Little,4,3,1,0,133.33,5
5,Cameron Green,c Shahrukh Khan b Little,1,2,0,0,50.00,1
6,Dinesh Karthik,not out,21,12,3,0,175.00,24
7,Swapnil Singh,not out,15,9,2,1,166.67,19
